In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ਪਦਾਅ 1: ਸੰਰਚਿਤ ਨਤੀਜਿਆਂ ਲਈ Pydantic ਮਾਡਲਾਂ ਦੀ ਪਰਿਭਾਸ਼ਾ ਦਿਓ

ਇਹ ਮਾਡਲ **schema** ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਦੇ ਹਨ ਜੋ ਏਜੰਟ ਵਾਪਸ ਕਰਨਗੇ। Pydantic ਨਾਲ `response_format` ਦੀ ਵਰਤੋਂ ਕਰਨ ਨਾਲ ਇਹ ਯਕੀਨੀ ਬਣਦਾ ਹੈ:
- ✅ ਟਾਈਪ-ਸੇਫ ਡਾਟਾ ਐਕਸਟ੍ਰੈਕਸ਼ਨ
- ✅ ਸਵੈਚਾਲਿਤ ਵੈਰੀਫਿਕੇਸ਼ਨ
- ✅ ਫ੍ਰੀ-ਟੈਕਸਟ ਜਵਾਬਾਂ ਤੋਂ ਕੋਈ ਪਾਰਸਿੰਗ ਗਲਤੀਆਂ ਨਹੀਂ
- ✅ ਖੇਤਰਾਂ ਦੇ ਆਧਾਰ 'ਤੇ ਆਸਾਨ ਸ਼ਰਤੀ ਰੂਟਿੰਗ


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ਕਦਮ 2: ਹੋਟਲ ਬੁਕਿੰਗ ਟੂਲ ਬਣਾਓ

ਇਹ ਟੂਲ ਉਹ ਹੈ ਜਿਸਨੂੰ **availability_agent** ਕਮਰੇ ਉਪਲਬਧ ਹਨ ਜਾਂ ਨਹੀਂ, ਇਹ ਚੈੱਕ ਕਰਨ ਲਈ ਕਾਲ ਕਰੇਗਾ। ਅਸੀਂ `@ai_function` ਡੈਕੋਰੇਟਰ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ ਤਾਂ ਜੋ:
- ਇੱਕ Python ਫੰਕਸ਼ਨ ਨੂੰ AI-ਕਾਲ ਕਰਨ ਯੋਗ ਟੂਲ ਵਿੱਚ ਬਦਲਿਆ ਜਾ ਸਕੇ
- LLM ਲਈ JSON schema ਆਟੋਮੈਟਿਕ ਤੌਰ 'ਤੇ ਤਿਆਰ ਕੀਤਾ ਜਾਵੇ
- ਪੈਰਾਮੀਟਰ ਵੈਲੀਡੇਸ਼ਨ ਨੂੰ ਹੈਂਡਲ ਕੀਤਾ ਜਾਵੇ
- ਏਜੰਟਾਂ ਦੁਆਰਾ ਆਟੋਮੈਟਿਕ ਕਾਲਿੰਗ ਨੂੰ ਸਮਰਥਿਤ ਕੀਤਾ ਜਾਵੇ

ਇਸ ਡੈਮੋ ਲਈ:
- **ਸਟਾਕਹੋਮ, ਸੀਐਟਲ, ਟੋਕੀਓ, ਲੰਡਨ, ਐਮਸਟਰਡਮ** → ਕਮਰੇ ਉਪਲਬਧ ਹਨ ✅
- **ਹੋਰ ਸਾਰੇ ਸ਼ਹਿਰ** → ਕਮਰੇ ਉਪਲਬਧ ਨਹੀਂ ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ਚਰਨ 3: ਰੂਟਿੰਗ ਲਈ ਸ਼ਰਤ ਫੰਕਸ਼ਨ ਪਰਿਭਾਸ਼ਿਤ ਕਰੋ

ਇਹ ਫੰਕਸ਼ਨ ਏਜੰਟ ਦੇ ਜਵਾਬ ਦੀ ਜਾਂਚ ਕਰਦੇ ਹਨ ਅਤੇ ਵਰਕਫਲੋ ਵਿੱਚ ਕਿਹੜਾ ਰਾਹ ਲੈਣਾ ਹੈ ਇਹ ਨਿਰਧਾਰਤ ਕਰਦੇ ਹਨ।

**ਮੁੱਖ ਪੈਟਰਨ:**
1. ਜਾਂਚ ਕਰੋ ਕਿ ਸੁਨੇਹਾ `AgentExecutorResponse` ਹੈ
2. ਸੰਰਚਿਤ ਆਉਟਪੁੱਟ (Pydantic ਮਾਡਲ) ਨੂੰ ਪਾਰਸ ਕਰੋ
3. ਰੂਟਿੰਗ ਨੂੰ ਨਿਯੰਤਰਿਤ ਕਰਨ ਲਈ `True` ਜਾਂ `False` ਵਾਪਸ ਕਰੋ

ਵਰਕਫਲੋ **edges** 'ਤੇ ਇਹ ਸ਼ਰਤਾਂ ਦਾ ਮੁਲਾਂਕਣ ਕਰੇਗਾ ਤਾਂ ਜੋ ਅਗਲੇ ਐਗਜ਼ਿਕਿਊਟਰ ਨੂੰ ਚੁਣਿਆ ਜਾ ਸਕੇ।


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## ਚੌਥਾ ਕਦਮ: ਕਸਟਮ ਡਿਸਪਲੇ ਐਗਜ਼ਿਕਿਊਟਰ ਬਣਾਓ

ਐਗਜ਼ਿਕਿਊਟਰ ਉਹ ਵਰਕਫਲੋ ਕੰਪੋਨੈਂਟ ਹਨ ਜੋ ਰੂਪਾਂਤਰਨ ਜਾਂ ਸਾਈਡ ਇਫੈਕਟਸ ਨੂੰ ਅੰਜਾਮ ਦਿੰਦੇ ਹਨ। ਅਸੀਂ `@executor` ਡੈਕੋਰੇਟਰ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ ਇੱਕ ਕਸਟਮ ਐਗਜ਼ਿਕਿਊਟਰ ਬਣਾਉਣ ਲਈ ਜੋ ਅੰਤਮ ਨਤੀਜਾ ਦਿਖਾਉਂਦਾ ਹੈ।

**ਮੁੱਖ ਧਾਰਨਾਵਾਂ:**
- `@executor(id="...")` - ਇੱਕ ਫੰਕਸ਼ਨ ਨੂੰ ਵਰਕਫਲੋ ਐਗਜ਼ਿਕਿਊਟਰ ਵਜੋਂ ਰਜਿਸਟਰ ਕਰਦਾ ਹੈ
- `WorkflowContext[Never, str]` - ਇਨਪੁਟ/ਆਉਟਪੁਟ ਲਈ ਟਾਈਪ ਹਿੰਟਸ
- `ctx.yield_output(...)` - ਅੰਤਮ ਵਰਕਫਲੋ ਨਤੀਜੇ ਨੂੰ ਯੀਲਡ ਕਰਦਾ ਹੈ


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## ਕਦਮ 5: ਵਾਤਾਵਰਣ ਵੈਰੀਏਬਲ ਲੋਡ ਕਰੋ

LLM ਕਲਾਇੰਟ ਨੂੰ ਕਨਫਿਗਰ ਕਰੋ। ਇਹ ਉਦਾਹਰਨ ਕੰਮ ਕਰਦੀ ਹੈ:
- **GitHub ਮਾਡਲਸ** (GitHub ਟੋਕਨ ਨਾਲ ਮੁਫ਼ਤ ਪੱਧਰ)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ਚਰਨ 6: ਸੰਰਚਿਤ ਨਤੀਜਿਆਂ ਨਾਲ AI ਏਜੰਟ ਬਣਾਓ

ਅਸੀਂ **ਤਿੰਨ ਵਿਸ਼ੇਸ਼ ਏਜੰਟ** ਬਣਾਉਂਦੇ ਹਾਂ, ਹਰ ਇੱਕ ਨੂੰ `AgentExecutor` ਵਿੱਚ ਲਪੇਟਿਆ ਜਾਂਦਾ ਹੈ:

1. **availability_agent** - ਟੂਲ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਹੋਟਲ ਦੀ ਉਪਲਬਧਤਾ ਦੀ ਜਾਂਚ ਕਰਦਾ ਹੈ
2. **alternative_agent** - ਵਿਕਲਪਕ ਸ਼ਹਿਰਾਂ ਦੀ ਸਲਾਹ ਦਿੰਦਾ ਹੈ (ਜਦੋਂ ਕੋਈ ਕਮਰੇ ਨਹੀਂ ਹੁੰਦੇ)
3. **booking_agent** - ਬੁਕਿੰਗ ਲਈ ਪ੍ਰੋਤਸਾਹਿਤ ਕਰਦਾ ਹੈ (ਜਦੋਂ ਕਮਰੇ ਉਪਲਬਧ ਹੁੰਦੇ ਹਨ)

**ਮੁੱਖ ਵਿਸ਼ੇਸ਼ਤਾਵਾਂ:**
- `tools=[hotel_booking]` - ਏਜੰਟ ਨੂੰ ਟੂਲ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ
- `response_format=PydanticModel` - ਸੰਰਚਿਤ JSON ਨਤੀਜਾ ਦੇਣ ਲਈ ਮਜਬੂਰ ਕਰਦਾ ਹੈ
- `AgentExecutor(..., id="...")` - ਵਰਕਫਲੋ ਵਿੱਚ ਵਰਤੋਂ ਲਈ ਏਜੰਟ ਨੂੰ ਲਪੇਟਦਾ ਹੈ


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ਕਦਮ 7: ਸ਼ਰਤੀ ਕਨੈਕਸ਼ਨਾਂ ਨਾਲ ਵਰਕਫਲੋ ਬਣਾਓ

ਹੁਣ ਅਸੀਂ `WorkflowBuilder` ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਸ਼ਰਤੀ ਰੂਟਿੰਗ ਨਾਲ ਗ੍ਰਾਫ ਬਣਾਉਂਦੇ ਹਾਂ:

**ਵਰਕਫਲੋ ਸੰਰਚਨਾ:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**ਮੁੱਖ ਤਰੀਕੇ:**
- `.set_start_executor(...)` - ਪ੍ਰਵੇਸ਼ ਬਿੰਦੂ ਸੈੱਟ ਕਰਦਾ ਹੈ
- `.add_edge(from, to, condition=...)` - ਸ਼ਰਤੀ ਕਨੈਕਸ਼ਨ ਜੋੜਦਾ ਹੈ
- `.build()` - ਵਰਕਫਲੋ ਨੂੰ ਅੰਤਮ ਰੂਪ ਦਿੰਦਾ ਹੈ


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## ਚਰਣ 8: ਟੈਸਟ ਕੇਸ 1 ਚਲਾਓ - ਸ਼ਹਿਰ ਬਿਨਾਂ ਉਪਲਬਧਤਾ (ਪੈਰਿਸ)

ਆਓ **ਕੋਈ ਉਪਲਬਧਤਾ ਨਹੀਂ** ਵਾਲੇ ਰਾਹ ਨੂੰ ਟੈਸਟ ਕਰੀਏ, ਪੈਰਿਸ ਵਿੱਚ ਹੋਟਲਾਂ ਦੀ ਬੇਨਤੀ ਕਰਕੇ (ਜਿਸ ਵਿੱਚ ਸਾਡੇ ਸਿਮੂਲੇਸ਼ਨ ਵਿੱਚ ਕੋਈ ਕਮਰੇ ਨਹੀਂ ਹਨ)।


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## ਚਰਨ 9: ਟੈਸਟ ਕੇਸ 2 ਚਲਾਓ - ਸ਼ਹਿਰ ਜਿਸ ਵਿੱਚ ਉਪਲਬਧਤਾ ਹੈ (ਸਟਾਕਹੋਮ)

ਹੁਣ ਆਓ **ਉਪਲਬਧਤਾ** ਦੇ ਰਾਹ ਨੂੰ ਟੈਸਟ ਕਰੀਏ, ਸਟਾਕਹੋਮ ਵਿੱਚ ਹੋਟਲਾਂ ਦੀ ਬੇਨਤੀ ਕਰਕੇ (ਜਿਸ ਵਿੱਚ ਸਾਡੇ ਸਿਮੂਲੇਸ਼ਨ ਵਿੱਚ ਕਮਰੇ ਹਨ)।


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## ਮੁੱਖ ਗੱਲਾਂ ਅਤੇ ਅਗਲੇ ਕਦਮ

### ✅ ਤੁਸੀਂ ਕੀ ਸਿੱਖਿਆ:

1. **WorkflowBuilder ਪੈਟਰਨ**
   - ਐਂਟਰੀ ਪਾਇੰਟ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਨ ਲਈ `.set_start_executor()` ਵਰਤੋ
   - ਸ਼ਰਤਾਂ ਦੇ ਅਧਾਰ 'ਤੇ ਰੂਟਿੰਗ ਲਈ `.add_edge(from, to, condition=...)` ਵਰਤੋ
   - ਵਰਕਫਲੋ ਨੂੰ ਅੰਤਮ ਰੂਪ ਦੇਣ ਲਈ `.build()` ਕਾਲ ਕਰੋ

2. **ਸ਼ਰਤਾਂ ਦੇ ਅਧਾਰ 'ਤੇ ਰੂਟਿੰਗ**
   - ਸ਼ਰਤਾਂ ਵਾਲੇ ਫੰਕਸ਼ਨ `AgentExecutorResponse` ਦੀ ਜਾਂਚ ਕਰਦੇ ਹਨ
   - ਰੂਟਿੰਗ ਫੈਸਲੇ ਕਰਨ ਲਈ ਸੰਰਚਿਤ ਆਉਟਪੁੱਟ ਨੂੰ ਪਾਰਸ ਕਰੋ
   - ਐਜ ਨੂੰ ਐਕਟੀਵੇਟ ਕਰਨ ਲਈ `True` ਵਾਪਸ ਕਰੋ, ਇਸਨੂੰ ਛੱਡਣ ਲਈ `False`

3. **ਟੂਲ ਇੰਟੀਗ੍ਰੇਸ਼ਨ**
   - Python ਫੰਕਸ਼ਨ ਨੂੰ AI ਟੂਲ ਵਿੱਚ ਬਦਲਣ ਲਈ `@ai_function` ਵਰਤੋ
   - ਏਜੰਟ ਜਦੋਂ ਲੋੜ ਹੋਵੇ ਤਾਂ ਟੂਲ ਨੂੰ ਆਪਣੇ ਆਪ ਕਾਲ ਕਰਦੇ ਹਨ
   - ਟੂਲ JSON ਵਾਪਸ ਕਰਦੇ ਹਨ ਜਿਸਨੂੰ ਏਜੰਟ ਪਾਰਸ ਕਰ ਸਕਦੇ ਹਨ

4. **ਸੰਰਚਿਤ ਆਉਟਪੁੱਟ**
   - ਟਾਈਪ-ਸੇਫ ਡਾਟਾ ਐਕਸਟ੍ਰੈਕਸ਼ਨ ਲਈ Pydantic ਮਾਡਲ ਵਰਤੋ
   - ਏਜੰਟ ਬਣਾਉਣ ਸਮੇਂ `response_format=MyModel` ਸੈਟ ਕਰੋ
   - ਜਵਾਬਾਂ ਨੂੰ `Model.model_validate_json()` ਨਾਲ ਪਾਰਸ ਕਰੋ

5. **ਕਸਟਮ ਐਗਜ਼ਿਕਿਊਟਰ**
   - ਵਰਕਫਲੋ ਕੰਪੋਨੈਂਟ ਬਣਾਉਣ ਲਈ `@executor(id="...")` ਵਰਤੋ
   - ਐਗਜ਼ਿਕਿਊਟਰ ਡਾਟਾ ਨੂੰ ਬਦਲ ਸਕਦੇ ਹਨ ਜਾਂ ਸਾਈਡ ਇਫੈਕਟ ਕਰ ਸਕਦੇ ਹਨ
   - ਵਰਕਫਲੋ ਨਤੀਜੇ ਪੈਦਾ ਕਰਨ ਲਈ `ctx.yield_output()` ਵਰਤੋ

### 🚀 ਅਸਲ ਦੁਨੀਆ ਵਿੱਚ ਵਰਤੋਂ:

- **ਟ੍ਰੈਵਲ ਬੁਕਿੰਗ**: ਉਪਲਬਧਤਾ ਦੀ ਜਾਂਚ ਕਰੋ, ਵਿਕਲਪ ਸੁਝਾਓ, ਵਿਕਲਪਾਂ ਦੀ ਤੁਲਨਾ ਕਰੋ
- **ਕਸਟਮਰ ਸਰਵਿਸ**: ਮਾਮਲੇ ਦੀ ਕਿਸਮ, ਭਾਵਨਾ, ਤਰਜੀਹ ਦੇ ਅਧਾਰ 'ਤੇ ਰੂਟ ਕਰੋ
- **ਈ-ਕਾਮਰਸ**: ਇਨਵੈਂਟਰੀ ਦੀ ਜਾਂਚ ਕਰੋ, ਵਿਕਲਪ ਸੁਝਾਓ, ਆਰਡਰ ਪ੍ਰੋਸੈਸ ਕਰੋ
- **ਕੰਟੈਂਟ ਮੋਡਰੇਸ਼ਨ**: ਟੌਕਸਿਸਿਟੀ ਸਕੋਰ, ਯੂਜ਼ਰ ਫਲੈਗ ਦੇ ਅਧਾਰ 'ਤੇ ਰੂਟ ਕਰੋ
- **ਅਪ੍ਰੂਵਲ ਵਰਕਫਲੋ**: ਰਕਮ, ਯੂਜ਼ਰ ਰੋਲ, ਰਿਸਕ ਲੈਵਲ ਦੇ ਅਧਾਰ 'ਤੇ ਰੂਟ ਕਰੋ
- **ਮਲਟੀ-ਸਟੇਜ ਪ੍ਰੋਸੈਸਿੰਗ**: ਡਾਟਾ ਦੀ ਗੁਣਵੱਤਾ, ਪੂਰਨਤਾ ਦੇ ਅਧਾਰ 'ਤੇ ਰੂਟ ਕਰੋ

### 📚 ਅਗਲੇ ਕਦਮ:

- ਹੋਰ ਜਟਿਲ ਸ਼ਰਤਾਂ ਸ਼ਾਮਲ ਕਰੋ (ਕਈ ਮਾਪਦੰਡ)
- ਵਰਕਫਲੋ ਸਟੇਟ ਮੈਨੇਜਮੈਂਟ ਨਾਲ ਲੂਪ ਲਾਗੂ ਕਰੋ
- ਦੁਬਾਰਾ ਵਰਤਣਯੋਗ ਕੰਪੋਨੈਂਟ ਲਈ ਸਬ-ਵਰਕਫਲੋ ਸ਼ਾਮਲ ਕਰੋ
- ਅਸਲ APIs (ਹੋਟਲ ਬੁਕਿੰਗ, ਇਨਵੈਂਟਰੀ ਸਿਸਟਮ) ਨਾਲ ਇੰਟੀਗ੍ਰੇਟ ਕਰੋ
- ਐਰਰ ਹੈਂਡਲਿੰਗ ਅਤੇ ਫਾਲਬੈਕ ਪਾਥ ਸ਼ਾਮਲ ਕਰੋ
- ਬਿਲਟ-ਇਨ ਵਿਜੁਅਲਾਈਜ਼ੇਸ਼ਨ ਟੂਲ ਨਾਲ ਵਰਕਫਲੋਜ਼ ਨੂੰ ਵਿਜੁਅਲਾਈਜ਼ ਕਰੋ



---

**ਅਸਵੀਕਰਤਾ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਹਾਲਾਂਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚੀਤਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦਸਤਾਵੇਜ਼ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਕਾਪੀ ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
